In [1]:

!pip install wget

import os
import re
import urllib.request
import urllib3
from bs4 import BeautifulSoup
import pandas as pd
import requests 
import shutil 
import wget
import time
import joblib

You should consider upgrading via the '/opt/virtual_env/py3/bin/python3 -m pip install --upgrade pip' command.


In [2]:
BASE_DIR = '../base_data'
ROOT_DIR = '../root_data'
NOTEBOOKS_DIR = 'notebooks'

# whitespace_patterns = re.compile(r'[\n\r\t]')
file_ext_pattern = r".*.pdf$"


In [3]:

def retrieve_documents_from_web(row:pd.core.series.Series) -> bool:

    dataRetrieved = False
    with urllib3.PoolManager() as http:
        try:
            resp = http.request('GET', row['urls'])
            with open(os.path.join(BASE_DIR,row['index']), 'wb') as fout:
                fout.write(resp.data)
            dataRetrieved = True
        except Exception as e:    
            dataRetrieved = False
            pass
    return dataRetrieved


In [23]:
crops_list = ['wheat','canola','barley','soybean','corn']

wheat_pest = ['Army worm','Gram pod borer','Pink stem borer','Brown mite','Aphids','Termites']

canola_pest = ['Alfalfa Looper', 'Autographia Californica Speyer', 'Aphids', 'Aster Leafhopper',
             'Beneficial Insects', 'Bertha Armyworm', 'Cabbage Seedpod Weevil', 'Cabbageworms',
             'Clover Cutworms', 'Cutworms', 'Diamondback moth', 'Flea Beetles', 'Lygus Bugs', 
             'Painted lady', 'Red turnip beetle', 'Swede midge']

corn_pest = ['Cutworms','Seed-Corn Maggot','Southern Corn Rootworm','Wireworms','Corn Leaf Aphid',
            'Corn Flea Beetle','Corn Earworm','Fall Armyworm','European Corn Borer','Corn (Dusky) Sap Beetle',
            'Japanese Beetles','Green Stink Bug']

soybean_pest = ['grasshoppers', 'green cloverworms', 'soybean loopers', 'Phomopsis spp', 'Cercospora kikuchii',
               'Sclerotinia sclerotiorum', 'soybean mosaic']

barley_pest = ['Sitophilus', 'Angoumois grain moth', 'Sitotroga cerealella', 'Sitophilus granarius', 'S. oryzae',
               'Sitophilus zeamais', 'African Armyworm']

pest_list = wheat_pest + canola_pest + corn_pest + soybean_pest + barley_pest
pest_list = list(set(pest_list))
print(f"Total pest {len(pest_list)}")
print(pest_list)

Total pest 46
['Corn Flea Beetle', 'Japanese Beetles', 'Swede midge', 'European Corn Borer', 'Clover Cutworms', 'Termites', 'Bertha Armyworm', 'Corn (Dusky) Sap Beetle', 'Green Stink Bug', 'Gram pod borer', 'Flea Beetles', 'Pink stem borer', 'Southern Corn Rootworm', 'Red turnip beetle', 'Beneficial Insects', 'Wireworms', 'soybean mosaic', 'Diamondback moth', 'Sitophilus', 'African Armyworm', 'Sitotroga cerealella', 'Cabbage Seedpod Weevil', 'Alfalfa Looper', 'Fall Armyworm', 'Sitophilus granarius', 'Army worm', 'Cabbageworms', 'Angoumois grain moth', 'soybean loopers', 'Cutworms', 'Aphids', 'Corn Earworm', 'grasshoppers', 'Phomopsis spp', 'Sclerotinia sclerotiorum', 'Aster Leafhopper', 'Corn Leaf Aphid', 'S. oryzae', 'Brown mite', 'Lygus Bugs', 'Painted lady', 'Seed-Corn Maggot', 'Autographia Californica Speyer', 'green cloverworms', 'Cercospora kikuchii', 'Sitophilus zeamais']


In [24]:
remove_pest_1 = ['Corn Flea Beetle']
for i in remove_pest_1:
    pest_list.remove(i)
    if i in pest_list: print(f"Found {i}")

len(pest_list)

45

In [25]:
pest_list_sample_1 = ['Japanese Beetles', 'Swede midge', 'European Corn Borer', 'Clover Cutworms', 'Termites']


In [5]:

def retrieve_images_from_web(query):
    """ Retrieve images 
    
    :param: image url
    """
    
    with urllib3.PoolManager() as http:
    
        response = http.request('GET', query)
        soup = BeautifulSoup(response.data)

        img_divs = soup.findAll('div', attrs={"class" : "col-xs-12 col-sm-6 col-md-4 text-center"})
        absoluteURLs, captions, img_names = [], [], []

        for div in img_divs:
            captions.append(div.text)
            for eachlink in div.find_all('img'):
                img_names.append(eachlink.get('alt'))
                absoluteURLs.append(eachlink.get('src'))

        if not os.path.exists(os.path.join(BASE_DIR,eachItem)):
            os.mkdir(os.path.join(BASE_DIR,eachItem))

        for image_name, image_url in zip(img_names,absoluteURLs):
            !wget "$image_url"
            print(f"Moving {image_name} to {os.path.join(BASE_DIR,eachItem)}")
            try:
                shutil.move(os.path.abspath(image_name+'.jpg'),os.path.join(BASE_DIR,eachItem))
            except Exception: pass
            time.sleep(2)
    
    joblib.dump(absoluteURLs, os.path.join(BASE_DIR,'url_'+eachItem+'.npy'))
    joblib.dump(captions, os.path.join(BASE_DIR,'caption_'+eachItem+'.npy'))
    joblib.dump(img_names, os.path.join(BASE_DIR,'img_name_'+eachItem+'.npy'))
    
    return absoluteURLs, captions, img_names


In [27]:
baseURL = 'https://www.ipmimages.org/search/action.cfm?q='

start = time.time()

for eachItem in pest_list_sample_1:
    print(f"FETCHING IMAGES FOR {eachItem}")
    query_token = eachItem.replace(' ','+')
    query1 = 'https://www.ipmimages.org/search/action.cfm?q='+query_token
#     query1 = 'https://www.ipmimages.org/search/action.cfm?q='+'ringtone'
    query2 = 'https://www.ipmimages.org/search/action.cfm?q='+query_token+'&start=201'

    absoluteURLs1, captions1, img_names1 = retrieve_images_from_web(query1)
    absoluteURLs2, captions2, img_names2 = retrieve_images_from_web(query2)

end = time.time()
print(f"Total time {end - start}")



FETCHING IMAGES FOR Japanese Beetles


/opt/virtual_env/py3/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


--2020-11-07 23:03:41--  https://bugwoodcloud.org/images/192x128/5539765.jpg
Resolving bugwoodcloud.org (bugwoodcloud.org)... 13.249.11.38, 13.249.11.56, 13.249.11.48, ...
Connecting to bugwoodcloud.org (bugwoodcloud.org)|13.249.11.38|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4733 (4.6K) [image/jpeg]
Saving to: ‘5539765.jpg’

5539765.jpg         100%[===================>]   4.62K  --.-KB/s    in 0s      

2020-11-07 23:03:42 (253 MB/s) - ‘5539765.jpg’ saved [4733/4733]

Moving 5539765 to ../base_data/Japanese Beetles
--2020-11-07 23:03:45--  https://bugwoodcloud.org/images/192x128/5539770.jpg
Resolving bugwoodcloud.org (bugwoodcloud.org)... 13.249.11.107, 13.249.11.48, 13.249.11.56, ...
Connecting to bugwoodcloud.org (bugwoodcloud.org)|13.249.11.107|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3192 (3.1K) [image/jpeg]
Saving to: ‘5539770.jpg’

5539770.jpg         100%[===================>]   3.12K  --.-KB/s    in 0s      

20

Moving 5569194 to ../base_data/Japanese Beetles
--2020-11-07 23:04:53--  https://bugwoodcloud.org/images/192x128/5596729.jpg
Resolving bugwoodcloud.org (bugwoodcloud.org)... 13.249.11.48, 13.249.11.38, 13.249.11.107, ...
Connecting to bugwoodcloud.org (bugwoodcloud.org)|13.249.11.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15153 (15K) [image/jpeg]
Saving to: ‘5596729.jpg’

5596729.jpg         100%[===================>]  14.80K  --.-KB/s    in 0.005s  

2020-11-07 23:04:55 (2.86 MB/s) - ‘5596729.jpg’ saved [15153/15153]

Moving 5596729 to ../base_data/Japanese Beetles
--2020-11-07 23:04:57--  https://bugwoodcloud.org/images/192x128/1243048.jpg
Resolving bugwoodcloud.org (bugwoodcloud.org)... 13.249.11.48, 13.249.11.38, 13.249.11.107, ...
Connecting to bugwoodcloud.org (bugwoodcloud.org)|13.249.11.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18088 (18K) [image/jpeg]
Saving to: ‘1243048.jpg’

1243048.jpg         100%[=======

Moving 5475437 to ../base_data/Japanese Beetles
--2020-11-07 23:05:42--  https://bugwoodcloud.org/images/192x128/5596652.jpg
Resolving bugwoodcloud.org (bugwoodcloud.org)... 13.249.11.48, 13.249.11.38, 13.249.11.107, ...
Connecting to bugwoodcloud.org (bugwoodcloud.org)|13.249.11.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16336 (16K) [image/jpeg]
Saving to: ‘5596652.jpg’

5596652.jpg         100%[===================>]  15.95K  --.-KB/s    in 0s      

2020-11-07 23:05:43 (97.9 MB/s) - ‘5596652.jpg’ saved [16336/16336]

Moving 5596652 to ../base_data/Japanese Beetles
--2020-11-07 23:05:45--  https://bugwoodcloud.org/images/192x128/5596920.jpg
Resolving bugwoodcloud.org (bugwoodcloud.org)... 13.249.11.38, 13.249.11.56, 13.249.11.107, ...
Connecting to bugwoodcloud.org (bugwoodcloud.org)|13.249.11.38|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8500 (8.3K) [image/jpeg]
Saving to: ‘5596920.jpg’

5596920.jpg         100%[=======

...

Moving 5467026 to ../base_data/Termites
--2020-11-08 00:58:07--  https://bugwoodcloud.org/images/192x128/5467027.jpg
Resolving bugwoodcloud.org (bugwoodcloud.org)... 13.249.11.48, 13.249.11.107, 13.249.11.38, ...
Connecting to bugwoodcloud.org (bugwoodcloud.org)|13.249.11.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3371 (3.3K) [image/jpeg]
Saving to: ‘5467027.jpg’

5467027.jpg         100%[===================>]   3.29K  --.-KB/s    in 0s      

2020-11-08 00:58:08 (838 MB/s) - ‘5467027.jpg’ saved [3371/3371]

Moving 5467027 to ../base_data/Termites
--2020-11-08 00:58:10--  https://bugwoodcloud.org/images/192x128/5467028.jpg
Resolving bugwoodcloud.org (bugwoodcloud.org)... 13.249.11.48, 13.249.11.107, 13.249.11.38, ...
Connecting to bugwoodcloud.org (bugwoodcloud.org)|13.249.11.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3582 (3.5K) [image/jpeg]
Saving to: ‘5467028.jpg’

5467028.jpg         100%[===================>]   3.

Moving 5467041 to ../base_data/Termites
--2020-11-08 00:58:56--  https://bugwoodcloud.org/images/192x128/5467042.jpg
Resolving bugwoodcloud.org (bugwoodcloud.org)... 13.249.11.48, 13.249.11.107, 13.249.11.38, ...
Connecting to bugwoodcloud.org (bugwoodcloud.org)|13.249.11.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2605 (2.5K) [image/jpeg]
Saving to: ‘5467042.jpg’

5467042.jpg         100%[===================>]   2.54K  --.-KB/s    in 0s      

2020-11-08 00:58:57 (662 MB/s) - ‘5467042.jpg’ saved [2605/2605]

Moving 5467042 to ../base_data/Termites
--2020-11-08 00:59:00--  https://bugwoodcloud.org/images/192x128/5467043.jpg
Resolving bugwoodcloud.org (bugwoodcloud.org)... 13.249.11.48, 13.249.11.107, 13.249.11.38, ...
Connecting to bugwoodcloud.org (bugwoodcloud.org)|13.249.11.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3417 (3.3K) [image/jpeg]
Saving to: ‘5467043.jpg’

5467043.jpg         100%[===================>]   3.

Moving 5467056 to ../base_data/Termites
--2020-11-08 00:59:45--  https://bugwoodcloud.org/images/192x128/5467057.jpg
Resolving bugwoodcloud.org (bugwoodcloud.org)... 13.249.11.56, 13.249.11.107, 13.249.11.38, ...
Connecting to bugwoodcloud.org (bugwoodcloud.org)|13.249.11.56|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4438 (4.3K) [image/jpeg]
Saving to: ‘5467057.jpg’

5467057.jpg         100%[===================>]   4.33K  --.-KB/s    in 0s      

2020-11-08 00:59:47 (872 MB/s) - ‘5467057.jpg’ saved [4438/4438]

Moving 5467057 to ../base_data/Termites
--2020-11-08 00:59:49--  https://bugwoodcloud.org/images/192x128/5467058.jpg
Resolving bugwoodcloud.org (bugwoodcloud.org)... 13.249.11.56, 13.249.11.107, 13.249.11.38, ...
Connecting to bugwoodcloud.org (bugwoodcloud.org)|13.249.11.56|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2955 (2.9K) [image/jpeg]
Saving to: ‘5467058.jpg’

5467058.jpg         100%[===================>]   2.

Moving 5467071 to ../base_data/Termites
--2020-11-08 01:00:34--  https://bugwoodcloud.org/images/192x128/5467072.jpg
Resolving bugwoodcloud.org (bugwoodcloud.org)... 13.249.11.56, 13.249.11.38, 13.249.11.107, ...
Connecting to bugwoodcloud.org (bugwoodcloud.org)|13.249.11.56|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3220 (3.1K) [image/jpeg]
Saving to: ‘5467072.jpg’

5467072.jpg         100%[===================>]   3.14K  --.-KB/s    in 0s      

2020-11-08 01:00:36 (785 MB/s) - ‘5467072.jpg’ saved [3220/3220]

Moving 5467072 to ../base_data/Termites
--2020-11-08 01:00:38--  https://bugwoodcloud.org/images/192x128/5467073.jpg
Resolving bugwoodcloud.org (bugwoodcloud.org)... 13.249.11.56, 13.249.11.38, 13.249.11.107, ...
Connecting to bugwoodcloud.org (bugwoodcloud.org)|13.249.11.56|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3956 (3.9K) [image/jpeg]
Saving to: ‘5467073.jpg’

5467073.jpg         100%[===================>]   3.

Moving 5467086 to ../base_data/Termites
--2020-11-08 01:01:24--  https://bugwoodcloud.org/images/192x128/5467087.jpg
Resolving bugwoodcloud.org (bugwoodcloud.org)... 13.249.11.38, 13.249.11.48, 13.249.11.107, ...
Connecting to bugwoodcloud.org (bugwoodcloud.org)|13.249.11.38|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2570 (2.5K) [image/jpeg]
Saving to: ‘5467087.jpg’

5467087.jpg         100%[===================>]   2.51K  --.-KB/s    in 0s      

2020-11-08 01:01:25 (502 MB/s) - ‘5467087.jpg’ saved [2570/2570]

Moving 5467087 to ../base_data/Termites
--2020-11-08 01:01:27--  https://bugwoodcloud.org/images/192x128/5467088.jpg
Resolving bugwoodcloud.org (bugwoodcloud.org)... 13.249.11.38, 13.249.11.48, 13.249.11.107, ...
Connecting to bugwoodcloud.org (bugwoodcloud.org)|13.249.11.38|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3482 (3.4K) [image/jpeg]
Saving to: ‘5467088.jpg’

5467088.jpg         100%[===================>]   3.

In [ ]:
print(len(absoluteURLs1), len(captions1), len(img_names1))
